![data-x](http://oi64.tinypic.com/o858n4.jpg)

---
# Cookbook 6: Basic MNIST in TensorFlow

**Author list:** Nathan Yan Cheng

**References / Sources:** 
* https://www.tensorflow.org/versions/r0.9/tutorials/mnist/beginners/index.html

**License Agreement:** Feel free to do whatever you want with this code

___

*In this notebook we will train a neural network to recognize handwritten digits*

## Welcome!

In this tutorial, you will be build and train a basic digit recognition network in TensorFlow. This doesn't quite count as "deep learning" yet, as we will not have any hidden layers in between the input and output layers. However, this model can be easily generalized to handle more complex neural network architectures.

First, we need to import the data we are going to work with. TensorFlow has some sample data ready for us.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

This sets "mnist" to an object that holds our data. We will see how to access this data later. Each training set holds an input and an output vector. The input vector is a flattened 28 x 28 image. The output vector is a "one_hot" vector (hence the one_hot setting above) that has ten entries, each one representing a digit. A one_hot vector is a vector common in classification that has a 1 in the correct entry, and 0s everywhere else. In our case, the one_hot vector representing 4 would be [0,0,0,1,0,0,0,0,0,0].

Now, let's import the actual tensorflow module.

In [ ]:
import tensorflow as tf

Next, we are going to initialize a variable for to represent the input. Each row is of length 28 * 28 = 728, the total number of pixels. Since we don't know how many rows we are going to have, we can leave the first dimension with size None. Note that this is mathematically fine since "x" is the leftmost matrix in any multiplication, thus leaving the number of rows invariant.

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])

TensorFlow works by first setting up the network architecture, then feeding in training data for training later. This "x" has no value, it simply represents where the inputs will go later. The "tf.float32" tells TensorFlow the promised type of the later inputs.

Now, let's create the weight and bias variables. The weight matrix will multiply "x" on the right and output a vector with ten entries, so it has dimensions 784 x 10. We want to add the bias vector to every row of this output. Fortunately, TensorFlow does this automatically for us if we simply use the additive notation, as we'll see later, so we can just give it size 1 x 10.

In [ ]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

A "variable" type in TensorFlow is something that TensorFlow will adjust the value of during training. We initialize all the values initially to zero with "tf.zeros."

Finally, we perform the multiplication, addition, and apply the softmax function to the output vector, and store the result in "y." You can look up the exact definition of the softmax function, but it is essentially a normalized logistic sigmoid function, and will "squash" the entries of the vector to add up to 1. This allows us to look at the output vector as a probability distribution over the possible digits 0-9.

In [ ]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

Note here that "xW" will give us a matrix of dimensions None x 10. Adding a vector of dimension 1 x 10 to this should make no sense, but TensorFlow automatically expands our "b" to be of size None x 10, thus adding "b" to every row.

Now, we'll create a placeholder object for the target output (which will hold the actual value when we are training). It gets dimensions None x 10, to match "y."

In [ ]:
y_ = tf.placeholder(tf.float32, [None, 10])

Now, we will define our loss function. We will be using cross entropy. This may be slightly different from cross entropy function you are familiar with. We first take the log of each element in our predicted vectors, then do a term by term multiplication of each entry with our target vectors. Then, we add up the entries of each vector using "reduce_mean." The "reduction_indices=[1]" tells TensorFlow to reduce "across" the vector instead "down" it. Then, we will negate it, since all the entries are between 0 and 1 and log will have produced very large negative numbers for entries farther from 1. Finally, we will average over the errors calculated for each vector to get our loss value.

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

You may notice that since "y_" will be a one hot vector, we will only be accounting for the error of one term, as everything else gets zeroed out. That means that we only care about minimizing the error of one entry: the entry in the correct digit position. At first, this may seem strange, but remember that we are using the softmax function, which normalizes all of our inputs to add up to 1. Making the correct value closer to 1 will, by extension, push the incorrect values closer to 0.

Now we define a training step. Each call to the training step should do one step of optimization. We will simply be doing gradient descent.

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

0.5 is our learning rate, and we place the loss function we want to minimize in the minimize() function.

Next, let's establish a way for us to more practically measure how well our model does. Obviously, we want our model to be correct more often than not, so let's look at the percentage of predictions we are getting right.

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

"tf.argmax" returns the position of the entry in a vector with the maximum value. Thus, argmax on [0.1, 0.6, 0.3] would return 1, as that is the index of 0.6. Calling argmax on our predicted vector basically gives us the entry with the "highest likelihood" of being the correct digit, if we view the vector as a probability distribution over the digits. "tf.equal" returns True if the entries are equal and False otherwise. Hence, "correct_prediction" gives us a list of booleans. To see what percentage we got correct, we simply cast the bools to floats (True = 1, False = 0), and average over the resulting vector.

Finally, we are ready to start training our network. We need to initialize all of the variables and start the session. This is mostly just TensorFlow convention.

In [ ]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Now, we run our "train_step" 1000 times on some data. We get this data by calling mnist.train.next_batch(100), which gives us 100 randomly selected data points, split respectively into our x values and our y values.

In [ ]:
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

Note how we run train_step. We don't just call train_step(), for instance. We need to feed data into the session. This is done through the feed dictionary, which will give the associated placholders the value you give them in the dictionary. Running a function in TensorFlow typically takes the form sess.run(function, feed_dict{placeholderName: actual_data}).

Lastly, we print our final accuracy. Note again the way we call the accuracy function.

In [ ]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))